## Imports

In [ ]:
import os

## Config

In [ ]:
feature_list_id = 'magic_jturkewitz'

In [ ]:
data_folder = os.path.abspath(os.path.join(os.curdir, os.pardir, 'data')) + os.path.sep
aux_data_folder = os.path.join(data_folder, 'aux') + os.path.sep
preproc_data_folder = os.path.join(data_folder, 'preproc') + os.path.sep
features_data_folder = os.path.join(data_folder, 'features') + os.path.sep

## Read Data

In [ ]:
df_questions_train = pd.read_csv(preproc_data_folder + 'train_phrasenorm.csv').fillna('')
df_questions_test = pd.read_csv(preproc_data_folder + 'test_phrasenorm.csv').fillna('')

In [ ]:
# df_questions_train = pd.read_csv(data_folder + 'train.csv').fillna('')
# df_questions_test = pd.read_csv(data_folder + 'test.csv').fillna('')

## Build Features

In [ ]:
train_orig = df_questions_train
test_orig = df_questions_test

In [ ]:
df1 = train_orig[['question1']].copy()
df2 = train_orig[['question2']].copy()
df1_test = test_orig[['question1']].copy()
df2_test = test_orig[['question2']].copy()

df2.rename(columns = {'question2':'question1'},inplace=True)
df2_test.rename(columns = {'question2':'question1'},inplace=True)

train_questions = df1.append(df2)
train_questions = train_questions.append(df1_test)
train_questions = train_questions.append(df2_test)
train_questions.drop_duplicates(subset = ['question1'],inplace=True)

train_questions.reset_index(inplace=True,drop=True)
questions_dict = pd.Series(train_questions.index.values,index=train_questions.question1.values).to_dict()

In [ ]:
train_questions

In [ ]:
questions_dict

In [ ]:
train_cp = train_orig.copy()
test_cp = test_orig.copy()
train_cp.drop(['qid1','qid2'],axis=1,inplace=True)

test_cp['is_duplicate'] = -1
test_cp.rename(columns={'test_id':'id'},inplace=True)
comb = pd.concat([train_cp,test_cp])

comb['q1_hash'] = comb['question1'].map(questions_dict)
comb['q2_hash'] = comb['question2'].map(questions_dict)

q1_vc = comb.q1_hash.value_counts().to_dict()
q2_vc = comb.q2_hash.value_counts().to_dict()

#map to frequency space
comb['q1_freq'] = comb['q1_hash'].map(lambda x: q1_vc.get(x, 0) + q2_vc.get(x, 0))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: q1_vc.get(x, 0) + q2_vc.get(x, 0))

# Calculate ratios
comb['freq_ratio'] = comb['q1_freq'] / comb['q2_freq']
comb['freq_ratio_reverse'] = comb['q2_freq'] / comb['q1_freq']

# # Hash differences
# comb['min_hash'] = np.min(comb[['q1_hash', 'q2_hash']], axis=1)
# comb['max_hash'] = np.max(comb[['q1_hash', 'q2_hash']], axis=1)
# comb['hash_diff'] = np.abs(comb['min_hash'] - comb['max_hash'])

# # Bin hashes
# comb['q1_hash'] = pd.cut(comb['q1_hash'], 100, labels=range(100))
# comb['q2_hash'] = pd.cut(comb['q2_hash'], 100, labels=range(100))

column_names = [
    'q1_freq',
    'q2_freq',
    'freq_ratio',
    'freq_ratio_reverse',
#     'q1_hash',
#     'q2_hash',
]

train_comb = comb[comb['is_duplicate'] >= 0][column_names]
test_comb = comb[comb['is_duplicate'] < 0][column_names]

# train_comb['hash_diff'] = train_comb['hash_diff'] / len(train_comb)
# test_comb['hash_diff'] = test_comb['hash_diff'] / len(test_comb)

In [ ]:
comb

## Save feature names

In [ ]:
feature_names = [
    'magic_jt_q1_freq',
    'magic_jt_q2_freq',
    'magic_jt_freq_ratio',
    'magic_jt_freq_ratio_reverse',
#     'magic_jt_hash_diff',
#     'magic_jt_q1_hash_bin',
#     'magic_jt_q2_hash_bin',
]

In [ ]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

## Save features

In [ ]:
X_train = train_comb.values.astype('float64')

In [ ]:
save(X_train, features_data_folder + f'X_train_{feature_list_id}.pickle')

In [ ]:
X_test = test_comb.values.astype('float64')

In [ ]:
save(X_test, features_data_folder + f'X_test_{feature_list_id}.pickle')